In [ ]:
import pandas as pd

def save_ids_to_txt_file(df, file_path):
    """
    Save the IDs from the DataFrame to a text file.
    """
    with open(file_path, 'w') as f:
        for index, row in df.iterrows():
            f.write(f"{row['id']}\n")
    
# Load the CSV file
df = pd.read_csv('data/h01.csv')

# Filter the neurons based on the 'data' column
df_neurons = df[df['data'].str.contains('#neuron')]
print("Neurons:", len(df_neurons))
save_ids_to_txt_file(df_neurons, 'neurons.ids')

# Filter the pyramidal cells based on the 'data' column
df_pyramidal = df_neurons[df_neurons['data'].str.contains('#pyramidal ')]
print("Pyramidals:", len(df_pyramidal))
save_ids_to_txt_file(df_pyramidal, 'pyramidal.ids')

# Get the puyramidal neurons in all layers 
for i in [1, 2, 3, 4, 5, 6]:
    df_pyramidal_layer = df_neurons[df_neurons['data'].str.contains(f'#L{i}')]
    print(f"Pyramidals: L{i}", len(df_pyramidal_layer))
    save_ids_to_txt_file(df_pyramidal_layer, f'pyramidal_L{i}.ids')
    

# Filter the interneuron cells based on the 'data' column
df_interneuron = df_neurons[df_neurons['data'].str.contains('#interneuron ')]
print("Interneurons:", len(df_interneuron))
save_ids_to_txt_file(df_interneuron, 'interneurons.ids')

for i in [1, 2, 3, 4, 5, 6]:
    df_interneuron_layer = df_interneuron[df_interneuron['data'].str.contains(f'#L{i}')]
    print(f"Interneurons: L{i}", len(df_interneuron_layer))
    save_ids_to_txt_file(df_interneuron_layer, f'interneurons_L{i}.ids')




Neurons: 15567
Pyramidals: 8626
Pyramidals: L1 555
Pyramidals: L2 4573
Pyramidals: L3 2639
Pyramidals: L4 3388
Pyramidals: L5 2268
Pyramidals: L6 1045
Interneurons: 687
Interneurons: L1 77
Interneurons: L2 253
Interneurons: L3 137
Interneurons: L4 94
Interneurons: L5 88
Interneurons: L6 25


In [65]:
import caveclient
url = "https://global.brain-wire-test.org/"
auth = caveclient.auth.AuthClient(server_address=url)
auth.setup_token(make_new=True)

New Tokens need to be acquired by hand. Please follow the following steps:
                1) Go to: https://global.brain-wire-test.org//auth/api/v1/create_token to create a new token.
                2) Log in with your Google credentials and copy the token shown afterward.
                3a) Save it to your computer with: client.auth.save_token(token="PASTE_YOUR_TOKEN_HERE")
                or
                3b) Set it for the current session only with client.auth.token = "PASTE_YOUR_TOKEN_HERE"
                Note: If you need to save or load multiple tokens, please read the documentation for details.
                Warning! Creating a new token by finishing step 2 will invalidate the previous token!


Opening in existing browser session.


In [67]:
auth_token = "be1bde27490bb4f37580cc370ba9068a"
auth.save_token(token=auth_token, overwrite=True)

In [ ]:
from caveclient import CAVEclient
client = CAVEclient()
try:
    client.auth.save_token(token=auth_token)
except :
    print("Token is already there!")

Token is already there!
8841fffdac923d4cee7b07fe82e11987


In [ ]:
import os 
import cloudvolume

def create_dir(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)


def get_cave_client(dataset):
    """
    get_cave_client Getting the CAVE client

    Args:
        dataset (_type_): Examples ['minnie65_public_v117', 'h01-release'] 
    """
    # Create the CAVE client
    client = caveclient.CAVEclient(dataset, auth_token=auth_token)
    
    # To access dynamic meshes, you can query the segmentation source from the info client
    # client.info.segmentation_source()
    print(client.info.get_datastack_info())
    
    # Return the client
    return client


def get_cloud_volume(path):
    """
    get_cloud_volume Getting the cloud volume

    Returns:
        _type_: Example: "precomputed://gs://iarpa_microns/minnie/minnie65/seg"
    """    
    # Return the cloud volume 
    print("Getting the cloud volume")
    return cloudvolume.CloudVolume(path, use_https=True, progress=True)


def load_ids_from_file(file_path):
    with open(file_path, 'r') as f:
        ids = list()
        for l in f:
            ids.append(l.strip('\n'))
    f.close()
    return ids

####################################################################################################
# @download_cell_from_cloud_volume
####################################################################################################
def download_cell_from_cloud_volume(cloud_volume, cell_id, output_directory):    

    try:
        # Load the mesh 
        mesh = cloud_volume.mesh.get(cell_id)[cell_id]

        # Write the mesh  
        # TODO: Add an option to save it as .h5 file 
        f = open('%s/%s.obj' % (output_directory, str(cell_id)), 'wb')
        f.write(mesh.to_obj())
        f.close()
    except Exception as e:
        print("Error in downloading the cell %s" % str(cell_id))
        print(e)
        return None


####################################################################################################
# @download_cells_from_cloud_volume
####################################################################################################  
def download_cells_from_cloud_volume(cloud_volume, cell_ids, output_directory):
    """
    download_cells_from_cloud_volume Downloading the cells from the cloud volume

    Args:
        cloud_volume (_type_): _description_
        cell_ids (_type_): _description_
        output_directory (_type_): _description_
    """    
    # Loop through the cell ids
    for cell_id in cell_ids:
        if (int(cell_id) > 0):
            print('Loading [%s]' % cell_id)
            download_cell_from_cloud_volume(cloud_volume, int(cell_id), output_directory)
        
        
def download_cells_from_cloud_volume_parallel(cloud_volume, cell_ids, output_directory, n_cores=8):
    """
    download_cells_from_cloud_volume Downloading the cells from the cloud volume

    Args:
        cloud_volume (_type_): _description_
        cell_ids (_type_): _description_
        output_directory (_type_): _description_
    """    
    
    commands = list()
    for i in cell_ids:
        commands.append([int(i), output_directory])
        
    from joblib import Parallel, delayed
    import multiprocessing
    Parallel(n_jobs=int(n_cores))(delayed(download_cell_from_cloud_volume)(
        cloud_volume, i[0], i[1]) for i in commands)

In [ ]:
ids_file = "/scratch/h01-data/ids/pyramidal_L5.ids"
output_directory = "/scratch/h01-data/meshes/pyramidal_L5"

ids = load_ids_from_file(ids_file)
print("Number of ids: ", len(ids))

# Create the output directory
create_dir(output_directory)

# Craete the cloud volume
cloud_volume_path = "precomputed://gs://h01-release/data/20210601/c3"
cloud_volume = get_cloud_volume(cloud_volume_path)

# Download the cells from the cloud volume
download_cells_from_cloud_volume(cloud_volume, ids, output_directory)